# OPEN WEATHER ONE CALL API FOR WEATHER VARIABLES
---
---

In [1]:
# https://openweathermap.org/api/one-call-3
# Lesson 5.01 - APIs
# https://note.nkmk.me/en/python-unix-time-datetime/

# Imports
---

In [2]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import requests

import datetime

%matplotlib inline
pd.set_option('display.max_columns', None)

# Testing: Current/2-Day Forecast Hourly Weather from One Call API
---

In [3]:
url_current_hourly = "https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,minutely,daily&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
current_hourly_req = requests.get(url_current_hourly)
print(f'Status: {current_hourly_req.status_code}')

w_current = current_hourly_req.json()
print(w_current.keys())

w_current_df = json_normalize(w_current['hourly'])
w_current_df = w_current_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point', 'uvi',
       'clouds', 'visibility', 'wind_speed', 'wind_deg', 'wind_gust']]
w_current_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'hourly'])


,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust
0,1659902400,305.24,304.19,1013,31,285.99,9.40,0,10000,2.49,212,2.40
1,1659906000,305.32,304.55,1013,33,287.02,8.89,0,10000,2.65,204,2.73
2,1659909600,305.83,304.76,1013,30,286.00,7.24,0,10000,2.92,202,2.97
3,1659913200,306.59,305.27,1012,27,285.05,4.98,0,10000,3.26,205,3.26
4,1659916800,307.13,305.52,1012,24,283.73,2.73,0,10000,4.24,201,3.54


In [4]:
# Convert dt UNIX to datetime UTC
w_current_df['dt'] = w_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [5]:
w_current_df.head()

,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust
0,2022-08-07 13:00:00,305.24,304.19,1013,31,285.99,9.40,0,10000,2.49,212,2.40
1,2022-08-07 14:00:00,305.32,304.55,1013,33,287.02,8.89,0,10000,2.65,204,2.73
2,2022-08-07 15:00:00,305.83,304.76,1013,30,286.00,7.24,0,10000,2.92,202,2.97
3,2022-08-07 16:00:00,306.59,305.27,1012,27,285.05,4.98,0,10000,3.26,205,3.26
4,2022-08-07 17:00:00,307.13,305.52,1012,24,283.73,2.73,0,10000,4.24,201,3.54


# Testing: Historical Daily Weather from One Call API
---

In [6]:
url_hist_daily = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.752125&lon=-121.288010&dt=1659470400&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
hist_daily_req = requests.get(url_hist_daily)
print(f'Status: {hist_daily_req.status_code}')

w_hist = hist_daily_req.json()
print(w_hist.keys())

w_hist_df = json_normalize(w_hist['data'])
w_hist_df = w_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'uvi', 'clouds', 'visibility', 'wind_speed', 'wind_deg']]
w_hist_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'data'])


,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg
0,1659470400,310.12,311.45,1013,32,290.7,9.41,0,10000,2.57,290


In [7]:
# Convert dt UNIX to datetime UTC
w_hist_df['dt'] = w_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [8]:
w_hist_df.head()

,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg
0,2022-08-02 13:00:00,310.12,311.45,1013,32,290.7,9.41,0,10000,2.57,290


In [9]:
w_hist['data'][0]['weather']

[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]

# Testing: Historical Hourly Weather from History API

In [12]:
url_hist_hourly = "http://history.openweathermap.org/data/2.5/history/city?lat=38.752125&lon=-121.288010&type=hour&start=1659470400&end=1659470400&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
hist_hourly_req = requests.get(url_hist_hourly)
print(f'Status: {hist_hourly_req.status_code}')

wh_hist = hist_hourly_req.json()
print(wh_hist.keys())

wh_hist_df = json_normalize(wh_hist['data'])
wh_hist_df = wh_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'uvi', 'clouds', 'visibility', 'wind_speed', 'wind_deg',
       'wind_gust']]
wh_hist_df.head()

Status: 401
dict_keys(['cod', 'message'])


KeyError: 'data'